In [ ]:
# use selenium


# open https://www.tradeport.xyz/?tab=trending


# find all the links to the NFTs


# GEt NFT Type for all the NFTs


# For example:https://www.tradeport.xyz/sui/collection/0xb07b09b016d28f989b6adda8069096da0c0a0ff6490f6e0866858c023b061bee%3A%3Amystic_yeti%3A%3AMysticYeti?bottomTab=trades&tab=items
# The type will be 0xb07b09b016d28f989b6adda8069096da0c0a0ff6490f6e0866858c023b061bee%3A%3Amystic_yeti%3A%3AMysticYeti

In [3]:
# Output the NFT types to a csv file

import pandas as pd

# Create an empty DataFrame with the specified columns
nft_df = pd.DataFrame(
    columns=["nft_collection_name", "nft_type", "verified", "tradeport_url"]
)


nft_df.to_csv("1_all_nft_types.csv", index=False)